In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery, bigquery_storage
from utils import *
import warnings 
warnings.filterwarnings('ignore')

import random
random.seed(12345)

In [2]:

# Join the zoominfo labels with the ga user features
query = """
    SELECT 
        * EXCEPT (user)
    FROM 
        `api-project-901373404215.lookalike_trail_data.zoominfo_user_feats_feb_april_training_v1` uf
    INNER JOIN
        `api-project-901373404215.lookalike_data.zoominfo_labels` labels
    USING
        (GA_fullVisitorId)
    """

In [3]:
## Fetching data
bqclient = bigquery.Client()
bqstorageclient = bigquery_storage.BigQueryReadClient()

df = bqclient.query(query).result().to_dataframe(bqstorage_client=bqstorageclient)
print(df.shape)


(250277, 192)


In [4]:
#lower case column names
df.columns= df.columns.str.lower()

In [5]:
#Creating new feature c_level
df['c_level']=np.where(df['managementlevel']=='C-Level',1,0)

In [6]:
df.c_level.value_counts(normalize=True)

0    0.759494
1    0.240506
Name: c_level, dtype: float64

In [7]:
#drop duplicates
df.drop_duplicates(subset=['ga_fullvisitorid','c_level'],inplace=True)
df.shape

(128394, 193)

In [8]:
# Upload to BQ
DATASET_ID = 'lookalike_trail_data'
TABLE_ID = 'feb_april_training_v1'   #change the name if there is any change in training dates

upload_bq(
    dataset_id=DATASET_ID,
    table_id=TABLE_ID,
    df=df,
    write_truncate=True # Will overwrite current table
)

feb_april_training_v1 loaded into BQ
